# Importando bibliotecas

In [ ]:
import pandas as pd
from tqdm import tqdm
import os
import time



# Classificação 1: Abrindo a base de dados filtrada por conceitos americanistas 

In [ ]:
df = pd.read_excel('seleção américa e variações.xlsx')
#ou df = pd.read_csv('seleção américa e variações.csv', encoding='latin1')



# Classificação, a partir das definições construídas com análise da bibliografia e documentação

In [ ]:
import os
import time
import pandas as pd
from tqdm import tqdm
import ollama          # pip install ollama

# ---- Função de classificação com Vicuna 7B ------------------------------- #
def classify_text_vicuna(segmento: str, model_name="vicuna:7b"):
    """
    Classifica um trecho de texto segundo as categorias solicitadas.
    Usa a interface OpenAI-like do Ollama para conversar com o modelo Vicuna.
    """
    system_prompt = """\
Você deve classificar trechos do texto abaixo usando ao menos uma ou mais \
das categorias da lista. A lista de categorias é:

1) Relações de poder autoritárias;
2) Discurso fascista;
3) Discurso golpista;
4) Justificativa para alguma forma de violência política;
5) Repressão;
6) Justificativa para desigualdades políticas;
7) Nenhuma das anteriores;

Para cada trecho, indique as categorias relevantes separadas por vírgulas. \
Exemplo de resposta: 1, 4
""".strip()

    while True:
        try:
            response = ollama.chat(
                model=model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user",   "content": segmento}
                ],
                temperature=0.0,
                top_p=1.0,
                # max_tokens equivalente → `options={"num_predict": 15}` se precisar
            )
            return response["message"]["content"].strip()

        # Erros específicos do Ollama
        except ollama.ResponseError as e:
            print(f"Ollama error: {e}. Retentando em 10 s…")
            time.sleep(10)

        # Qualquer outra exceção inesperada
        except Exception as e:
            print(f"Erro inesperado: {e}. Retentando em 10 s…")
            time.sleep(10)

# ---- Leitura do CSV ------------------------------------------------------ #
try:
    df = pd.read_csv("unificada.csv", on_bad_lines="skip")
except Exception as e:
    raise SystemExit(f"Falha ao carregar CSV: {e}")

# ---- Continua de onde parou --------------------------------------------- #
last_index_file = "last_index.txt"
class_file      = "data_class.csv"

last_index = int(open(last_index_file).read()) if os.path.isfile(last_index_file) else 0
if os.path.isfile(class_file):
    df_class = pd.read_csv(class_file, on_bad_lines="skip")
else:
    df_class = df.copy()
    df_class["classification"] = ""

total_rows = len(df)

# ---- Loop principal ------------------------------------------------------ #
for i in tqdm(range(last_index, total_rows), initial=last_index, total=total_rows):
    seg = df.loc[i, "Segmento"]
    df_class.loc[i, "classification"] = classify_text_vicuna(seg)

    # Salvar progresso periódico
    if i % 100 == 0 or i == total_rows - 1:
        df_class.to_csv(class_file, index=False)
        with open(last_index_file, "w") as f:
            f.write(str(i))

        processed = i + 1
        remaining = total_rows - processed
        print(f"✔ {processed}/{total_rows} processados — {remaining} restantes")


# Classificação 2: Abrindo a base de dados filtrada por conceitos coloniais

- Abrindo base de dados

In [ ]:
df = pd.read_excel('class_colony.xlsx')
#ou df = pd.read_csv('seleção américa e variações.csv', encoding='latin1')



# Código de classificação com definição a partir da bibliografia e documentação

In [ ]:
import os
import time
import pandas as pd
from tqdm import tqdm
import ollama          # pip install ollama

# --------------------------------------------------------------------- #
# 1. Função de classificação usando Vicuna 7B                           #
# --------------------------------------------------------------------- #
def classify_text_vicuna(segmento: str, model_name: str = "vicuna:7b") -> str:
    """Classifica o trecho como 'Com violência' ou 'Sem violência'."""
    system_prompt = """\
Classifique o uso do conceito de colônia no texto abaixo como 'Com violência'
ou 'Sem violência'.

- Definição de violência: uso intencional da força física, psicológica, 
simbólica ou do poder, de maneira ameaçadora ou real, contra pessoa, grupo 
ou comunidade, causando ou podendo causar lesões, coerção, morte, danos 
psicológicos, malformações ou privações. Inclui atos diretos (agressão física) 
ou indiretos (discriminação, opressão sistêmica) em contextos doméstico, 
político, social ou cultural.

Responda **somente** com 'Com violência' ou 'Sem violência'.
""".strip()

    while True:
        try:
            resp = ollama.chat(
                model=model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user",   "content": segmento}
                ],
                temperature=0.0,
                top_p=1.0,
                # se precisar limitar tokens: options={"num_predict": 10}
            )
            return resp["message"]["content"].strip()

        except ollama.ResponseError as e:
            print(f"Ollama error: {e}. Aguardando 10 s…")
            time.sleep(10)

        except Exception as e:
            print(f"Erro inesperado: {e}. Aguardando 10 s…")
            time.sleep(10)

# --------------------------------------------------------------------- #
# 2. Carregar dados e retomar onde parou                                #
# --------------------------------------------------------------------- #
try:
    df = pd.read_csv("unificada.csv", on_bad_lines="skip")
except Exception as e:
    raise SystemExit(f"Falha ao carregar CSV: {e}")

last_index_file = "last_index.txt"
class_file      = "data_class.csv"

last_index = int(open(last_index_file).read()) if os.path.isfile(last_index_file) else 0

if os.path.isfile(class_file):
    df_class = pd.read_csv(class_file, on_bad_lines="skip")
else:
    df_class = df.copy()
    df_class["classification"] = ""

total_rows = len(df)

# --------------------------------------------------------------------- #
# 3. Loop principal                                                     #
# --------------------------------------------------------------------- #
for i in tqdm(range(last_index, total_rows), initial=last_index, total=total_rows):
    seg = df.loc[i, "Segmento"]
    df_class.loc[i, "classification"] = classify_text_vicuna(seg)

    # Salva a cada 100 linhas (ou no último registro)
    if i % 100 == 0 or i == total_rows - 1:
        df_class.to_csv(class_file, index=False)
        with open(last_index_file, "w") as f:
            f.write(str(i))

        processed  = i + 1
        remaining  = total_rows - processed
        print(f"✔ {processed}/{total_rows} processados — {remaining} restantes")


# Classificação 3: Abrindo a base de dados filtrada por conceitos de autoridade/poder

In [ ]:
df = pd.read_excel('seleção I Congresso.xlsx')
#ou df = pd.read_csv('seleção américa e variações.csv', encoding='latin1')



# Código de classificação

In [ ]:
import os
import time
import pandas as pd
from tqdm import tqdm
import ollama          # pip install ollama

# ------------------------------------------------------------------ #
# 1. Função de classificação                                         #
# ------------------------------------------------------------------ #
def classify_text_vicuna(trecho: str, model_name: str = "vicuna:7b") -> str:
    """
    Classifica um trecho segundo as categorias 1–7.
    Retorna algo como '1, 4' ou '7', de acordo com instruções do prompt.
    """
    system_prompt = """\
Você deve classificar trechos do texto abaixo usando ao menos uma ou mais
das categorias da lista. A lista de categorias é:

1) Relações de poder autoritárias;
2) Discurso fascista;
3) Discurso golpista;
4) Justificativa para alguma forma de violência política;
5) Repressão;
6) Justificativa para desigualdades políticas;
7) Nenhuma das anteriores;

Para cada trecho, indique as categorias relevantes separadas por vírgulas.
Exemplo de resposta: 1, 4
""".strip()

    while True:
        try:
            resp = ollama.chat(
                model=model_name,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user",   "content": trecho}
                ],
                temperature=0.0,
                top_p=1.0,
                # Caso precise limitar tokens: options={"num_predict": 10}
            )
            return resp["message"]["content"].strip()

        except ollama.ResponseError as e:
            print(f"Ollama error: {e}. Aguardando 10 s…")
            time.sleep(10)

        except Exception as e:
            print(f"Erro inesperado: {e}. Aguardando 10 s…")
            time.sleep(10)

# ------------------------------------------------------------------ #
# 2. Carrega CSV e retoma índice                                     #
# ------------------------------------------------------------------ #
try:
    df = pd.read_csv("unificada.csv", on_bad_lines="skip")
except Exception as e:
    raise SystemExit(f"Falha ao carregar CSV: {e}")

last_index_file = "last_index.txt"
class_file      = "data_class.csv"

last_index = int(open(last_index_file).read()) if os.path.isfile(last_index_file) else 0

if os.path.isfile(class_file):
    df_class = pd.read_csv(class_file, on_bad_lines="skip")
else:
    df_class = df.copy()
    df_class["classification"] = ""

total_rows = len(df)

# ------------------------------------------------------------------ #
# 3. Loop principal                                                  #
# ------------------------------------------------------------------ #
for i in tqdm(range(last_index, total_rows), initial=last_index, total=total_rows):
    seg = df.loc[i, "Segmento"]
    df_class.loc[i, "classification"] = classify_text_vicuna(seg)

    # Salva a cada 100 registros ou no final
    if i % 100 == 0 or i == total_rows - 1:
        df_class.to_csv(class_file, index=False)
        with open(last_index_file, "w") as f:
            f.write(str(i))

        processed  = i + 1
        remaining  = total_rows - processed
        print(f"✔ {processed}/{total_rows} processados — {remaining} restantes")
